Evaluate the models

# Stochastic Adversarial Training (StochAT)

### SoTA

vanila SGD: 
MNIST - 99%+ (most cnns), CIFAR10 - 93%+ (resnet18), 96%+ (wideresnet) 

MNIST:

adversarial attacks: 
l-inf @ eps = 80/255 @20 steps: TRADES - 96.07% - (4 layer cnn), MART 96.4%, MMA 95.5%, PGD - 96.01% - (4 layer cnn)

adversarial attacks:
l-2 @ eps = 32/255 (check): TRADES, MMA, PGD

CIFAR10:

adversarial attacks: 
l-inf @ eps = 8/255 @20 steps: 
TRADES 53-56% - (WRN-34-10), MART 57-58% (WRN-34-10), MMA 47%, PGD 48% - (WRN-32-10)// 49% - (WRN-34-10), Std - 0.03%
https://openreview.net/pdf?id=rklOg6EFwS (Table 4)

adversarial attacks: 
l-inf @ eps = 8/255 @20 steps: 
[ResNet10] TRADES 45.4%, MART 46.6%, MMA 37.26%, PGD 42.27%, Std 0.14%

Benign accuracies: TRADES 84.92%, MART 83.62%, MMA 84.36, PGD 87.14%, Std 95.8% [wideresnet]
https://openreview.net/pdf?id=Ms9zjhVB5R (Table 1)

adversarial attacks:
l-2 @ eps = 32/255 (check): TRADES, MART, MMA, PGD

TBD: CWinf attacks

## Pretrained models for comparison

download pretrained models and place in ../trainedmodels/MNIST or ../trainedmodels/CIFAR10 respectively

### TRADES :
https://github.com/yaodongyu/TRADES (MNIST: small cnn, CIFAR10: WideResNet34)
### MMA : 
https://github.com/BorealisAI/mma_training (MNIST: lenet5, CIFAR10: WideResNet28)
### MART :
 https://github.com/YisenWang/MART (CIFAR10: ResNet18 and WideResNet34)

## IMPORT LIBRARIES

In [1]:
import numpy as np
import pandas as pd
from torch import nn, optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import transforms, datasets
from multiprocessing import cpu_count
from collections import OrderedDict
import matplotlib.pyplot as plt
import torch
import olympic
from typing import Union, Callable, Tuple
import sys
sys.path.append('../adversarial/')
sys.path.append('../architectures/')
from functional import boundary, iterated_fgsm, local_search, pgd, entropySmoothing
from ESGD_utils import *
import pickle
import time
import torch.backends.cudnn as cudnn
import argparse, math, random
import ESGD_optim
from trades import trades_loss

In [2]:
from torch.autograd import Variable

In [3]:
from torch.optim.lr_scheduler import StepLR

In [4]:
if torch.cuda.is_available():
    DEVICE = 'cuda'
else:
    DEVICE = 'cpu'

In [5]:
DEVICE

'cuda'

# LOAD DATA

In [6]:
#place data folders outside working directory

In [7]:
kwargs = {'num_workers': 1, 'pin_memory': True}

In [35]:
args = {}
args['test_batch_size'] = 128
args['no_cuda'] = False
args['epsilon'] = 0.031
args['num_steps'] = 20
args['step_size'] = 0.003
args['random'] =True,
args['white_box_attack']=True
args['log_interval'] = 100
args['beta'] = 1.0
args['seed'] = 1
args['lr'] = 0.1
args['momentum'] = 0.9
args['epochs'] = 5
args['batch_size'] = 128
args['save_freq'] = 5

In [9]:
dataset = 'CIFAR10' # [MNIST, CIFAR10]
transform = transforms.Compose([
    transforms.ToTensor(),
])
bsz = 128
if dataset == 'MNIST':
    train = datasets.MNIST('../../data/MNIST', train=True, transform=transform, download=True)
    val = datasets.MNIST('../../data/MNIST', train=False, transform=transform, download=True)
elif dataset == 'CIFAR10':
    train = datasets.CIFAR10('../../data/CIFAR10', train=True, transform=transform, download=True)
    val = datasets.CIFAR10('../../data/CIFAR10', train=False, transform=transform, download=True)
    
train_loader = DataLoader(train, batch_size=128, shuffle=True, **kwargs)
val_loader = DataLoader(val, batch_size=128, shuffle=False, **kwargs)

Files already downloaded and verified
Files already downloaded and verified


# INITIALIZE NETWORK

In [10]:
if dataset=='MNIST':
    from small_cnn import SmallCNN   
    Net = SmallCNN
    NetName = 'SmallCNN'

In [11]:
if dataset=='CIFAR10':
    #[ResNet18,ResNet34,ResNet50,WideResNet]
    from resnet import ResNet18,ResNet34,ResNet50
    from wideresnet import WideResNet
    Net = WideResNet
    NetName = 'WideResNet'

# RANDOM SEED 

In [12]:
seed = 1
torch.set_num_threads(2)
if DEVICE=='cuda':
    torch.cuda.set_device(-1)
    torch.cuda.manual_seed(seed)
    cudnn.benchmark = True
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# LOAD PRETRAINED OR TRAIN NEW MODELS:

In [12]:
TrainSGD = False
TrainESGD = False
TrainL2 = False
TrainLInf = False
TrainSAT2 = False
TrainSATInf = True
TrainTRADES = False
TrainTRADESInf = False
# tbd: add training modules for MART(inf only), MMA and MMAInf - always keep False
TrainMART = False 
TrainMMA = False
TrainMMAInf = False

# TRAIN NAIVE MODEL USING SGD

In [13]:
def eval_train(model, device, train_loader):
    model.eval()
    train_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            train_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    train_loss /= len(train_loader.dataset)
    print('Training: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))
    training_accuracy = correct / len(train_loader.dataset)
    return train_loss, training_accuracy

In [14]:
def eval_test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('Test: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_accuracy = correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [15]:
def evaluate_against_adversary(model, k, eps, step, norm):
    total = 0
    acc = 0
    for x, y in val_loader:
        total += x.size(0)
        
        if norm == 2:
            x_adv = pgd(
                model, x.to(DEVICE), y.to(DEVICE), torch.nn.CrossEntropyLoss(), k=k, step=step, eps=eps, norm=2)
            #print('rel. l2-norm of x_adv-x:',torch.norm(x_adv.detach().cpu()-x)/np.sqrt(x.size(0)))#/torch.norm(x))
            if torch.norm(x_adv.detach().cpu()-x)/np.sqrt(x.size(0))<0.6*eps :
                print('reconfigure attack')
            elif torch.norm(x_adv.detach().cpu()-x)/np.sqrt(x.size(0))>eps :
                print('reconfigure attack')    
                break;
        elif norm == 'inf':
            x_adv = iterated_fgsm(
                model, x.to(DEVICE), y.to(DEVICE), torch.nn.CrossEntropyLoss(), k=k, step=step, eps=eps, norm='inf')
            #print('rel. linf-norm of x_adv-x:',infnorm(x_adv.detach().cpu()-x)/infnorm(x))
            if infnorm(x_adv.detach().cpu()-x)/infnorm(x)<0.75*eps:
                print('reconfigure attack')
                break;
        y_pred = model(x_adv)

        acc += olympic.metrics.accuracy(y.to(DEVICE), y_pred) * x.size(0)
    print('Adversarial Accuracy: {}/{} ({:.0f}%)'.format(acc,total,100. * acc / total))
    return acc/total

In [36]:
def _pgd_whitebox(model,
                  X,
                  y,
                  epsilon=args['epsilon'],
                  num_steps=args['num_steps'],
                  step_size=args['step_size']):
    out = model(X)
    err = (out.data.max(1)[1] != y.data).float().sum()
    X_pgd = Variable(X.data, requires_grad=True)
    if args['random']:
        random_noise = torch.FloatTensor(*X_pgd.shape).uniform_(-epsilon, epsilon).to(DEVICE)
        X_pgd = Variable(X_pgd.data + random_noise, requires_grad=True)

    for _ in range(num_steps):
        opt = optim.SGD([X_pgd], lr=1e-3)
        opt.zero_grad()

        with torch.enable_grad():
            loss = nn.CrossEntropyLoss()(model(X_pgd), y)
        loss.backward()
        eta = step_size * X_pgd.grad.data.sign()
        X_pgd = Variable(X_pgd.data + eta, requires_grad=True)
        eta = torch.clamp(X_pgd.data - X.data, -epsilon, epsilon)
        X_pgd = Variable(X.data + eta, requires_grad=True)
        X_pgd = Variable(torch.clamp(X_pgd, 0, 1.0), requires_grad=True)
    err_pgd = (model(X_pgd).data.max(1)[1] != y.data).float().sum()
    #print('err pgd (white-box): ', err_pgd)
    return err, err_pgd

def eval_adv_test_whitebox(model, device, test_loader):
    """
    evaluate model by white-box attack
    """
    model.eval()
    robust_err_total = 0
    natural_err_total = 0

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        # pgd attack
        X, y = Variable(data, requires_grad=True), Variable(target)
        err_natural, err_robust = _pgd_whitebox(model, X, y)
        robust_err_total += err_robust
        natural_err_total += err_natural
    print('natural_acc_total: ', 100-natural_err_total.item()/100)
    print('robust_acc_total: ', 100-robust_err_total.item()/100)

In [17]:
def train_sgd(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        ypred = model(data)
        
        optimizer.zero_grad()
        sgd_loss = nn.CrossEntropyLoss()
        # calculate robust loss
        loss = sgd_loss(ypred,target)
        
        #print('outloss pre step:',loss)
        loss.backward(retain_graph=True)
        
        optimizer.step()
        #print('outloss post step:',loss.item())

        # print progress
        print_progress = False
        if print_progress and batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [18]:
if TrainSGD:
    ## initialize model
    model_SGD = Net().to(DEVICE)
    ## training params
    epochs = 20
    
    ## train model
    model_SGD = Net().to(DEVICE)
    ## training params
    lr = 0.01
    optimizer = optim.SGD(model_SGD.parameters(), lr=lr, momentum=0.9)
    scheduler = StepLR(optimizer, step_size=15, gamma=0.5)    
    ## train model

    for epoch in range(1, epochs + 1):
        print('Epoch:',epoch)
        # adjust learning rate for SGD

        # adversarial training
        train_sgd(model_SGD, DEVICE, train_loader, optimizer, epoch)
        # evaluation on natural examples
        print('================================================================')
        eval_train(model_SGD, DEVICE, train_loader)
        eval_test(model_SGD, DEVICE, val_loader)
        if (epoch) % 5 == 0:
            eval_adv_test_whitebox(model_SGD, DEVICE, val_loader)        
        print('================================================================')
        scheduler.step()

    ## save model
    modelname = '../trainedmodels/'+dataset+'/'+NetName+'_SGD_ep'+str(epochs)+'_lr'+str(lr)+'.pt'
    torch.save(model_SGD,modelname)

# TRAIN MODEL USING ENTROPY SGD (ESGD)

In [19]:
def entropy_training(model, optimiser, loss_fn, x, y, epoch):   
    model.train()
    y_pred = model(x)
    
    def helper():
        def feval():
            #x, y = Variable(x), Variable(y.squeeze())
            bsz = x.size(0)
            optimiser.zero_grad()
            yh = model(x)
            f = loss_fn.forward(yh, y)
            f.backward()

            yp = yh.argmax(axis=1)
            prec1 = 100*torch.sum(yp == y)//bsz
            err = 100.-prec1.item()

            return (f.data.item(), err)
        return feval

    loss, err = optimiser.step(helper(), model, loss_fn)
    loss = torch.tensor(loss)
    return loss, y_pred

In [20]:
def train_entropy(model, device, train_loader, optimizer, epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        ypred = model(data)
        
        sgd_loss = nn.CrossEntropyLoss()
        # calculate robust loss
        loss = entropy_training(model,optimizer,sgd_loss,data,target,epoch)

In [21]:
if TrainESGD:
    ## initialize model
    model_ESGD = Net().to(DEVICE)
    ## training parameters
    lr = 0.01 
    l2 = 0.0 #l2 regularization
    L = 0    #langevin iterations
    gamma = 1e-4 
    scoping = 1e-3
    noise = 1e-4
    loss_fn = nn.CrossEntropyLoss()
    epochs = 20
    optimiser = ESGD_optim.EntropySGD(model_ESGD.parameters(),config = dict(lr=lr, momentum=0.9, nesterov=True, weight_decay=l2,
            L=L, eps=noise, g0=gamma, g1=scoping))

    scheduler = StepLR(optimiser, step_size=20, gamma=0.5)    
    ## train model

    for epoch in range(1, epochs + 1):
        print('Epoch:',epoch)
        # adjust learning rate for SGD

        # adversarial training
        train_entropy(model_ESGD, DEVICE, train_loader, optimiser, epoch)
        # evaluation on natural examples
        print('================================================================')
        eval_train(model_ESGD, DEVICE, train_loader)
        eval_test(model_ESGD, DEVICE, val_loader)
        if (epoch) % 5 == 0:
            eval_adv_test_whitebox(model_ESGD, DEVICE, val_loader)        

            #evaluate_against_adversary(model_ESGD, k=20, eps=3.0, step=0.5, norm=2)
        print('================================================================')
        scheduler.step()
    ## save trained model
    modelname = '../trainedmodels/'+dataset+'/'+NetName+'_ESGD_ep'+str(epochs)+'_lr'+str(lr)+'.pt'
    torch.save(model_ESGD,modelname)

# TRAIN MODEL USING PGD SGD 

In [22]:
def infnorm(x):
    infn = torch.max(torch.abs(x.detach().cpu()))
    return infn

In [23]:
def adversarial_training(model, optimiser, loss_fn, x, y, epoch, adversary, k, step, eps, norm, random):
    """Performs a single update against a specified adversary"""
    model.train()

    # Adversial perturbation
    x_adv = adversary(model, x, y, loss_fn, k=k, step=step, eps=eps, norm=norm, random=True)
    #print('l2:',torch.norm(x_adv.detach().cpu()-x.detach().cpu())/np.sqrt(x.detach().cpu().size(0)))    
    #print('linf:',infnorm(x_adv.detach().cpu()-x.detach().cpu())/infnorm(x))    

    optimiser.zero_grad()
    y_pred = model(x_adv)
    loss = loss_fn(y_pred, y)
    loss.backward()
    optimiser.step()

    return loss, y_pred

In [24]:
def train_adversarial(method,model, device, train_loader, optimizer, epoch,adversary,k,step,eps,norm,random):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        ypred = model(data)
        
        sgd_loss = nn.CrossEntropyLoss()
        # calculate robust loss
        loss = method(model,optimizer,sgd_loss,data,target,epoch,adversary,k,step,eps,norm,random)

## l2 ball

In [25]:
if TrainL2:
    ## initialize model
    model_l2 = Net().to(DEVICE)
    
    ## training params
    epochs = 15
    lr = 0.01
    optimizer = optim.SGD(model_l2.parameters(), lr=lr,momentum=0.9)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.5)    
    ## train model

    for epoch in range(1, epochs + 1):
        print('Epoch:',epoch)
        # adjust learning rate for SGD

        # adversarial training
        train_adversarial(adversarial_training,model_l2, DEVICE, train_loader, optimizer, epoch,adversary=pgd,k=10,step=3,eps=3.0,norm=2,random=True)
        # evaluation on natural examples
        print('================================================================')
        eval_train(model_l2, DEVICE, train_loader)
        eval_test(model_l2, DEVICE, val_loader)
        if (epoch) % 5 == 0:
            eval_adv_test_whitebox(model_l2, DEVICE, val_loader)        
            #evaluate_against_adversary(model_l2, k=10, eps=3.0, step=3, norm=2)
        print('================================================================')
        scheduler.step()

    ## save trained model
    modelname = '../trainedmodels/'+dataset+'/'+NetName+'_AT2_ep'+str(epochs)+'_lr'+str(lr)+'.pt'
    torch.save(model_l2,modelname)

## linf ball

In [26]:
if TrainLInf:
    ## initialize model
    model_linf = Net().to(DEVICE)

    ## training params
    epochs = 35
    lr = 0.01
    optimizer = optim.SGD(model_linf.parameters(), lr=lr,momentum=0.9)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.5)    
    ## train model

    for epoch in range(1, epochs + 1):
        print('Epoch:',epoch)
        # adjust learning rate for SGD

        # adversarial training
        train_adversarial(adversarial_training,model_linf, DEVICE, train_loader, optimizer, epoch,adversary=iterated_fgsm,k=40,step=0.01,eps=0.3,norm='inf',random=False)
        # evaluation on natural examples
        print('================================================================')
        eval_train(model_linf, DEVICE, train_loader)
        eval_test(model_linf, DEVICE, val_loader)
        if (epoch) % 5 == 0:
            eval_adv_test_whitebox(model_linf, DEVICE, val_loader)        

            #evaluate_against_adversary(model_linf, k=40, eps=0.3, step=0.01, norm='inf')
        print('================================================================')
        scheduler.step()
        
    ## save trained model
    modelname = '../trainedmodels/'+dataset+'/'+NetName+'_ATInf_ep'+str(epochs)+'_lr'+str(lr)+'.pt'
    torch.save(model_linf,modelname)

# TRAIN MODEL USING SAT

In [44]:
def adversarial_training_entropy(model, optimiser, loss_fn, x, y, epoch, adversary, k, step, eps, norm, random):
    """Performs a single update against a specified adversary"""
    model.train()
    
    # Adversial perturbation
    #alpha = 0.5
    N = 2
    loss = 0
    for l in range(N):
        x_adv = adversary(model, x, y, loss_fn, k=k, step=step, step2=0.05, eps=eps, norm=norm, random=random, gamma=0.3, ep=1e-3,projector=False, debug=False)
        #print('l2:',torch.norm(x_adv.detach().cpu()-x.detach().cpu())/np.sqrt(x.detach().cpu().size(0)))    
        #print('linf:',infnorm(x_adv.detach().cpu()-x.detach().cpu())/infnorm(x))          
        optimiser.zero_grad()
        y_pred = model(x_adv)
        loss = loss + loss_fn(y_pred,y)
        #loss = (1-alpha)*loss + alpha*loss_fn(y_pred, y)
    loss = loss/N
    loss.backward()
    optimiser.step()
    return loss, y_pred

In [29]:
if TrainSAT2:
    ## initialize model
    model_SAT2 = Net().to(DEVICE)

    ## training params
    epochs = 15
    lr = 0.1
    optimizer = optim.SGD(model_SAT2.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.5)    
    ## train model

    for epoch in range(1, epochs + 1):
        print('Epoch:',epoch)
        # adjust learning rate for SGD

        # adversarial training
        train_adversarial(adversarial_training_entropy,model_SAT2, DEVICE, train_loader, optimizer, epoch,adversary=entropySmoothing,k=10,step=0.2,eps=3.0,norm=2,random=True)
        # evaluation on natural examples
        print('================================================================')
        eval_train(model_SAT2, DEVICE, train_loader)
        eval_test(model_SAT2, DEVICE, val_loader)
        if (epoch) % 5 == 0:
            evaluate_against_adversary(model_SAT2, k=10, eps=3.0, step=3, norm=2)
        print('================================================================')
        scheduler.step()
    ## save model
    modelname = '../trainedmodels/'+dataset+'/'+NetName+'_SAT2_ep'+str(epochs)+'_lr'+str(lr)+'.pt'
    torch.save(model_SAT2,modelname)

In [29]:
model_dir = '../WRN'

In [45]:
if TrainSATInf:
    ## initialize model
    #model_SATInf = Net().to(DEVICE)

    ## training params
    epochs = 10
    lr = 0.1 #0.03-96.65`
    optimizer = optim.SGD(model_SATInf.parameters(), lr=lr, momentum=0.9)
    scheduler = StepLR(optimizer, step_size=40, gamma=0.1)    
    ## train model

    for epoch in range(81, epochs + 81):
        print('Epoch:',epoch)
        # adjust learning rate for SGD
#10,0.05,0.3
        # adversarial training
        train_adversarial(adversarial_training_entropy,model_SATInf, DEVICE, train_loader, optimizer, epoch,adversary=entropySmoothing,k=10,step=0.007,eps=0.031,norm='inf',random=False)
        #train_adversarial(adversarial_training,model_SATInf, DEVICE, train_loader, optimizer, epoch,adversary=iterated_fgsm,k=40,step=0.01,eps=0.3,norm='inf',random=False)
        
        # evaluation on natural examples

        if (epoch) % 5 == 0:
            print('================================================================')
            #eval_train(model_SATInf, DEVICE, train_loader)
            #eval_test(model_SATInf, DEVICE, val_loader)
            eval_adv_test_whitebox(model_SATInf, DEVICE, val_loader)            
            print('================================================================')
        scheduler.step()
        

         # save checkpoint
        if epoch % args['save_freq'] == 0:
            torch.save(model_SATInf.state_dict(),
                       os.path.join(model_dir, 'model-nn-epoch{}.pt'.format(epoch)))
            torch.save(optimizer.state_dict(),
                       os.path.join(model_dir, 'opt-nn-checkpoint_epoch{}.tar'.format(epoch)))
    ## save model
    modelname = '../trainedmodels/'+dataset+'/'+NetName+'_SATInf_ep'+str(epochs+80)+'_lr'+str(lr)+'.pt'
    torch.save(model_SATInf,modelname)

Epoch: 81
Epoch: 82
Epoch: 83
Epoch: 84
Epoch: 85
natural_acc_total:  65.75
robust_acc_total:  36.47
Epoch: 86
Epoch: 87
Epoch: 88
Epoch: 89
Epoch: 90
natural_acc_total:  65.84
robust_acc_total:  35.849999999999994


# TRAIN MODEL USING TRADES

In [37]:
#model = SmallCNN().to(device)
#model.load_state_dict(torch.load('../../TRADES/model-mnist-smallCNN/model-nn-epoch20.pt'))
#model.load_state_dict(torch.load('../smallCNNtrained/model-nn-epoch5.pt'))

In [ ]:
def train_trades(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()

        # calculate robust loss
        loss = trades_loss(model=model,
                           x_natural=data,
                           y=target,
                           optimizer=optimizer,
                           step_size=args['step_size'],
                           epsilon=args['epsilon'],
                           perturb_steps=args['num_steps'],
                           beta=args['beta'])

        loss.backward()
        optimizer.step()

        # print progress
        #if batch_idx % args['log_interval'] == 0:
        #    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #        epoch, batch_idx * len(data), len(train_loader.dataset),
        #        100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
def adjust_learning_rate(optimizer, epoch):
    """decrease the learning rate"""
    lr = args['lr']
    if epoch >= 55:
        lr = args['lr'] * 0.1
    if epoch >= 75:
        lr = args['lr'] * 0.01
    if epoch >= 90:
        lr = args['lr'] * 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
if TrainTRADES:
    args['attack'] = 'l_2' #or 'l_inf'
    ## initialize model
    model_TRADES = model_cnn().to(DEVICE)
    ## training params
    lr = 0.1
    optimizer = optim.SGD(model_TRADES.parameters(), lr=lr)
    epochs = 10
    ## train model

    for epoch in range(1, epochs + 1):
        # adjust learning rate for SGD
        #adjust_learning_rate(optimizer, epoch)

        # adversarial training
        train_trades(args, model_TRADES, DEVICE, train_loader, optimizer, epoch)

        # evaluation on natural examples
        print('================================================================')
        eval_train(model_TRADES, DEVICE, train_loader)
        eval_test(model_TRADES, DEVICE, val_loader)
        evaluate_against_adversary(model_SATInf, k=40, eps=0.3, step=0.01, norm='inf')

        print('================================================================')

    ## save model
    modelname = '../trainedmodels/'+dataset+'/'+NetName+'_TRADES_ep'+str(epochs)+'_lr'+str(lr)+'.pt'
    torch.save(model_TRADES,modelname)

In [ ]:
if TrainTRADESInf:
    args['attack'] = 'l_inf' 
    ## initialize model
    model_TRADESInf = SmallCNN().to(DEVICE)
    ## training params
    lr = 0.01
    optimizer = optim.SGD(model_TRADESInf.parameters(), lr=lr, momentum=0.9)
    epochs = 5
    ## train model

    for epoch in range(1, epochs + 1):
        print('Epoch:',epoch)
        # adjust learning rate for SGD
        adjust_learning_rate(optimizer, epoch)

        # adversarial training
        train_trades(args, model_TRADESInf, DEVICE, train_loader, optimizer, epoch)

        # evaluation on natural examples
        if epoch % 5 == 0:
            print('================================================================')
            #eval_train(model_TRADESInf, DEVICE, train_loader)
            #eval_test(model_TRADESInf, DEVICE, val_loader)
            #evaluate_against_adversary(model_TRADESInf, k=40, eps=0.3, step=0.01, norm='inf')
            eval_adv_test_whitebox(model_TRADESInf, DEVICE, val_loader)
            print('================================================================')

    ## save model
    modelname = '../trainedmodels/'+dataset+'/'+NetName+'_TRADESInf_ep'+str(epochs)+'_lr'+str(lr)+'.pt'
    torch.save(model_TRADESInf,modelname)

# TRAIN MODEL USING MART

# TRAIN MODEL USING MMA

In [ ]:
#TBD

# LOAD ALL PRE-TRAINED MODELS

In [ ]:
TrainSGD = TrainSGD*True
TrainESGD = TrainESGD*True
TrainL2 = TrainL2*True
TrainLInf = TrainLInf*True
TrainSAT2 = TrainSAT2*True
TrainSATInf = TrainSATInf*True
TrainTRADES = TrainTRADES*True
TrainTRADESInf = TrainTRADESInf*True
TrainMART = TrainMART*False
TrainMMA = TrainMMA*False
TrainMMAInf = TrainMMAInf*False

In [ ]:
TrainTRADESInf = True

In [ ]:
# Load all the pre-trained models
if not TrainSGD:
    model_SGD = torch.load('../trainedmodels/'+dataset+'/SGD_ep10_lr0.1.pt').to(DEVICE)
if not TrainESGD:    
    model_ESGD = torch.load('../trainedmodels/'+dataset+'/ESGD_ep5_lr0.1.pt').to(DEVICE)
if not TrainLInf:
    adv_model_linf = torch.load('../trainedmodels/'+dataset+'/AT2_ep2_lr0.1.pt').to(DEVICE)
if not TrainL2:
    adv_model_l2 = torch.load('../trainedmodels/'+dataset+'/ATInf_ep2_lr0.1.pt').to(DEVICE)
if not TrainSAT2:
    model_SAT2 = torch.load('../trainedmodels/'+dataset+'/SAT2_ep2_lr0.1.pt').to(DEVICE)
if not TrainSATInf:
    model_SATInf = torch.load('../trainedmodels/'+dataset+'/SATInf_ep2_lr0.1.pt').to(DEVICE)
if not TrainTRADES:
    model_TRADES = torch.load('../trainedmodels/'+dataset+'/TRADES_ep2_lr0.1.pt').to(DEVICE)
if not TrainTRADESInf:
    model_TRADES = torch.load('../trainedmodels/'+dataset+'/TRADESInf_ep2_lr0.1.pt').to(DEVICE)    
if not TrainMART:
    if dataset=='CIFAR10':
        model_MART = Net()
        if NetName == 'ResNet18':
            dics = torch.load('../trainedmodels/'+dataset+'/ResNet18_model_MART.pt')
        if NetName == 'WideResNet':
            dics = torch.load('../trainedmodels/'+dataset+'/rob_cifar_mart.pt')            
        model_MART.load_state_dict(dics)
        model_MART = model_MART.to(DEVICE)
if not TrainMMA:
    model_MMA = torch.load('../trainedmodels/'+dataset+'/'+dataset.lower()+'-L2-MMA-4.0-sd0/model_best.pt')
    model_MMAInf = torch.load('../trainedmodels/'+dataset+'/'+dataset.lower()+'-Linf-MMA-0.45-sd0/model_best.pt')

## Quantifying adversarial accuracy

In [ ]:
def infnorm(x):
    infn = torch.max(torch.abs(x.detach().cpu()))
    return infn

## Evaluate robust models

In [ ]:
loadResults = False

In [ ]:
t1 = time.time()
if not loadResults:
    pgd_attack_range = [3.0] #np.arange(0, 6.1, 1./3)
    acc_SGD = []
    acc_ESGD = []
    acc_l2 = []
    acc_SAT2 = []
    acc_TRADES = []
    acc_MMA = []
    for eps in pgd_attack_range:
        print('eps:',eps)
        print('evaluating SGD network...')
        acc_SGD.append(evaluate_against_adversary(model_SGD, k=20, eps=eps, step=0.5, norm=2))
        print('evaluating ESGD network...')
        acc_ESGD.append(evaluate_against_adversary(model_ESGD, k=20, eps=eps, step=0.5, norm=2))
        print('evaluating SAT2 network...')
        acc_SAT2.append(evaluate_against_adversary(model_SAT2, k=30, eps=eps, step=0.3, norm=2))
        print('evaluating l2 network...')
        acc_l2.append(evaluate_against_adversary(adv_model_l2, k=30, eps=eps, step=1.5, norm=2))
        print('evaluating TRADES network...')
        acc_TRADES.append(evaluate_against_adversary(model_TRADES, k=30, eps=eps, step=1.5, norm=2))  
        print('evaluating MART network...')
        acc_MMA.append(evaluate_against_adversary(model_MMA, k=30, eps=eps, step=1.5, norm=2))          
print("time elapsed:",time.time()-t1)    

In [ ]:
if not loadResults:    
    accData = [acc_SGD,acc_ESGD,acc_l2,acc_SAT2,acc_TRADES,acc_MMA]
    np.save('../results/accData_l2.npy',accData)

In [ ]:
if loadResults:
    pgd_attack_range = [3.0] #np.arange(0.0, 6.1, 1./3)
    accData2 = np.load('../results/accData_l2.npy')
    [acc_SGD,acc_ESGD,acc_l2,acc_SAT2,acc_TRADES,acc_MMA] = accData2

In [ ]:
with plt.style.context('ggplot'):
    COLOURS = plt.rcParams['axes.prop_cycle'].by_key()['color']
    MARKERS = plt.rcParams['axes.prop_cycle'].by_key()['marker']

In [ ]:
if len(pgd_attack_range) > 1:
    with plt.style.context('ggplot'):
        plt.rcParams["font.weight"] = "bold"
        plt.rcParams["axes.labelweight"] = "bold"
        plt.rcParams["axes.labelcolor"] = "black"
        fig, axes = plt.subplots(1, 1, figsize=(15,10))
        plt.figure(figsize=(14,7))
        axes.set_title('$L^2$-bounded adversary')
        axes.plot(pgd_attack_range, acc_SGD, label='SGD')
        axes.plot(pgd_attack_range, acc_ESGD, label='Entropy-SGD')
        axes.plot(pgd_attack_range, acc_SAT2, label='Data-Entropy-SGD ($L_2$)')
        axes.plot(pgd_attack_range, acc_l2, label='$L2$ training')
        axes.plot(pgd_attack_range, acc_TRADES, label='TRADES training')
        axes.plot(pgd_attack_range, acc_MMA, label='MMA training')

        axes.vlines([3], 0, 1, colors=COLOURS[1], linestyle='--')
        axes.set_ylabel('Accuracy')
        axes.set_xlabel('Epsilon')
        axes.set_ylim((0,1))
        axes.legend()
else:
    print(acc_Data)

In [ ]:
loadResultsFGSM = False

In [ ]:
t1 = time.time()
if not loadResultsFGSM:
    fgsm_attack_range = [8/255] #np.arange(0.0, 0.52, 0.025)
    fgsm_acc_linf = []
    fgsm_acc_SGD = []
    fgsm_acc_ESGD = []
    fgsm_acc_SATInf = []
    fgsm_acc_TRADESInf = []
    fgsm_acc_MART = []
    fgsm_acc_MMAInf = []
    for eps in fgsm_attack_range:
        print('eps:',eps)
        print('evaluating SGD network...')
        fgsm_acc_SGD.append(evaluate_against_adversary(model_SGD, k=20, eps=eps, step=0.1, norm='inf'))
        print('evaluating ESGD network...')
        fgsm_acc_ESGD.append(evaluate_against_adversary(model_ESGD, k=20, eps=eps, step=0.1, norm='inf'))
        print('evaluating SAT2 network...')
        fgsm_acc_SATInf.append(evaluate_against_adversary(model_SATInf, k=10, eps=eps, step=0.1, norm='inf'))        
        print('evaluating linf network...')
        fgsm_acc_linf.append(evaluate_against_adversary(adv_model_linf, k=50, eps=eps, step=0.02, norm='inf'))
        print('evaluating TRADES network...')
        fgsm_acc_TRADESInf.append(evaluate_against_adversary(model_TRADESInf, k=50, eps=eps, step=0.02, norm='inf'))
        print('evaluating MART network...')
        fgsm_acc_MART.append(evaluate_against_adversary(model_MART, k=50, eps=eps, step=0.02, norm='inf'))
        print('evaluating MMA network...')
        fgsm_acc_MMAInf.append(evaluate_against_adversary(model_MMAInf, k=50, eps=eps, step=0.02, norm='inf'))
         
print("time elapsed:",time.time()-t1)    

In [ ]:
if not loadResultsFGSM:    
    fgsmaccData = [fgsm_acc_SGD,fgsm_acc_ESGD,fgsm_acc_linf,fgsm_acc_SATInf, fgsm_acc_TRADESInf, fgsm_acc_MART, fgsm_acc_MMAInf]
    np.save('../results/fgsmaccData.npy',fgsmaccData)

In [ ]:
if loadResultsFGSM:
    fgsm_attack_range = [8/255] #np.arange(0.0, 0.52, 0.025)
    accData2 = np.load('../results/fgsmaccData.npy')
    [fgsm_acc_SGD,fgsm_acc_ESGD,fgsm_acc_linf,fgsm_acc_SATInf, fgsm_acc_TRADESInf, fgsm_acc_MART, fgsm_acc_MMAInf] = accData2

In [ ]:
if len(fgsm_attack_range) > 1:
    with plt.style.context('ggplot'):
        plt.rcParams["font.weight"] = "bold"
        plt.rcParams["axes.labelweight"] = "bold"
        plt.rcParams["axes.labelcolor"] = "black"
        fig, axes = plt.subplots(1, 1, figsize=(15,10))
        plt.figure(figsize=(14,7))
        axes.set_title('$L^\infty$-bounded adversary')
        axes.plot(fgsm_attack_range, fgsm_acc_SGD, label='SGD')
        axes.plot(fgsm_attack_range, fgsm_acc_ESGD, label='Entropy-SGD')
        axes.plot(fgsm_attack_range, fgsm_acc_SATInf, label='Data-Entropy-SGD ($L_\infty$)')    
        axes.plot(fgsm_attack_range, fgsm_acc_linf, label='$L{\infty}$ training')
        axes.plot(fgsm_attack_range, fgsm_acc_TRADES, label='TRADES training')
        axes.vlines([0.25], 0, 1, colors=COLOURS[1], linestyle='--')
        axes.set_ylabel('Accuracy')
        axes.set_xlabel('Epsilon')
        axes.set_ylim((0,1))
        axes.legend()
else:
    print(acc_Data)